<a href="https://colab.research.google.com/github/natadekoko68/signate/blob/984/%E8%87%AA%E5%8B%95%E8%BB%8A%E3%81%AE%E8%B5%B0%E8%A1%8C%E8%B7%9D%E9%9B%A2%E4%BA%88%E6%B8%AC(%E6%8F%90%E5%87%BA19_40).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("content/")

Mounted at content/


In [75]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
import numpy as np
import warnings
warnings.simplefilter('ignore')

In [93]:
train = pd.read_csv("/content/content/MyDrive/Signate_data/2308自動車の走行距離予測/train.csv")
test = pd.read_csv("/content/content/MyDrive/Signate_data/2308自動車の走行距離予測/test.csv")
sample_submit = pd.read_csv("/content/content/MyDrive/Signate_data/2308自動車の走行距離予測/sample_submit.csv",header=None)
train_len = len(train.reset_index(drop=True))
df = pd.concat([train,test],axis=0)
df["car name"] = df["car name"].str.split().apply(lambda x: x[0])
df.loc[df["car name"]=='vokswagen', "car name"] = 'volkswagen'
df.loc[df["car name"]=='vw', "car name"] = 'volkswagen'
df.loc[df["car name"]=='toyouta', "car name"] = 'toyota'

car_manufacturer_country = {
    'amc': 'USA',
    'audi': 'Germany',
    'buick': 'USA',
    'capri': 'USA',
    'chevrolet': 'USA',
    'chevy': 'USA',
    'datsun': 'Japan',
    'dodge': 'USA',
    'fiat': 'Italy',
    'ford': 'USA',
    'hi': 'USA',
    'honda': 'Japan',
    'mercury': 'USA',
    'oldsmobile': 'USA',
    'opel': 'Germany',
    'peugeot': 'France',
    'plymouth': 'USA',
    'pontiac': 'USA',
    'renault': 'France',
    'saab': 'Sweden',
    'toyota': 'Japan',
    'triumph': 'UK',
    'volkswagen': 'Germany',
    'volvo': 'Sweden'
}
df["country"] = df["car name"].map(car_manufacturer_country)

df.loc[df["horsepower"]=="?","horsepower"]  = df.loc[df["horsepower"]!="?","horsepower"].astype(int).mean()
df = df.drop(["id"],axis=1)
# df["model year_qcut"] = pd.qcut(df["model year"],6,labels=False)
# df["model year_qcut"] = pd.qcut(df["model year"],6,labels=False)

df = pd.get_dummies(df, columns=['country',"car name","cylinders","origin"], prefix=['country',"car name","cylinders","origin"]).reset_index(drop=True)
# encoded_df = pd.get_dummies(df, columns=["car name"], prefix=["car name"]).reset_index(drop=True)

temp = df.loc[:train_len-1,]
X = temp.drop(["mpg"],axis=1)
y = temp["mpg"]
test = df.loc[train_len:,:]
test = test.drop(["mpg"],axis=1)


In [94]:
df

,mpg,displacement,horsepower,weight,acceleration,model year,country_France,country_Germany,country_Italy,country_Japan,...,car name_toyota,car name_triumph,car name_volkswagen,car name_volvo,cylinders_4,cylinders_6,cylinders_8,origin_1,origin_2,origin_3
0,23.059782,140,110,2815,17.977429,80,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1,17.674521,350,150,4456,13.514535,72,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
2,17.136353,302,140,2774,13.209912,79,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
3,22.664666,400,85,2190,15.196381,71,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
4,17.872018,429,220,2245,9.621400,70,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,225,105,3870,15.376311,76,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
996,NaN,140,112,2720,13.616843,82,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
997,NaN,318,110,2774,13.272636,75,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
998,NaN,90,75,2807,17.792560,77,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [100]:
for i in range(100):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=i)
    forest = RandomForestRegressor(n_estimators=100,
                                #    criterion='',
                                max_depth=None,
                                min_samples_split=2,
                                min_samples_leaf=1,
                                min_weight_fraction_leaf=0.0,
                                max_leaf_nodes=None,
                                min_impurity_decrease=0.0,
                                max_features="sqrt",
                                bootstrap=True,
                                oob_score=False,
                                n_jobs=-1,
                                random_state=42,
                                verbose=0,
                                warm_start=False,
                                ccp_alpha=0.0,
                                max_samples=None
                                )
    forest.fit(X_train, y_train)
    y_train_pred = forest.predict(X_train)
    y_test_pred  = forest.predict(X_valid)

    print('RMSE 学習: %.2f, テスト: %.2f' % (
            mean_squared_error(y_train, y_train_pred, squared=False), # 学習
            mean_squared_error(y_valid, y_test_pred, squared=False)    # テスト
        ))

RMSE 学習: 1.08, テスト: 3.33
RMSE 学習: 1.09, テスト: 2.98
RMSE 学習: 1.10, テスト: 2.91
RMSE 学習: 1.11, テスト: 2.64
RMSE 学習: 1.08, テスト: 2.95
RMSE 学習: 1.16, テスト: 2.69
RMSE 学習: 1.12, テスト: 3.17
RMSE 学習: 1.11, テスト: 2.60
RMSE 学習: 1.15, テスト: 3.23
RMSE 学習: 1.12, テスト: 2.93
RMSE 学習: 1.11, テスト: 2.97
RMSE 学習: 1.11, テスト: 2.47
RMSE 学習: 1.11, テスト: 3.15
RMSE 学習: 1.15, テスト: 2.51
RMSE 学習: 1.09, テスト: 3.28
RMSE 学習: 1.15, テスト: 2.50
RMSE 学習: 1.09, テスト: 2.60
RMSE 学習: 1.16, テスト: 2.77
RMSE 学習: 1.13, テスト: 3.25
RMSE 学習: 1.07, テスト: 3.58
RMSE 学習: 1.05, テスト: 3.52
RMSE 学習: 1.11, テスト: 3.28
RMSE 学習: 1.15, テスト: 2.59
RMSE 学習: 1.09, テスト: 2.83
RMSE 学習: 1.12, テスト: 2.57
RMSE 学習: 1.04, テスト: 3.18
RMSE 学習: 1.09, テスト: 2.57
RMSE 学習: 1.13, テスト: 2.69
RMSE 学習: 1.11, テスト: 2.55
RMSE 学習: 1.14, テスト: 2.61
RMSE 学習: 1.16, テスト: 2.26
RMSE 学習: 1.09, テスト: 2.88
RMSE 学習: 1.09, テスト: 3.54
RMSE 学習: 1.08, テスト: 2.85
RMSE 学習: 1.08, テスト: 3.05
RMSE 学習: 1.16, テスト: 2.69
RMSE 学習: 1.04, テスト: 3.14
RMSE 学習: 1.08, テスト: 3.06
RMSE 学習: 1.06, テスト: 3.14
RMSE 学習: 1.12, テスト: 2.38


In [101]:
forest = RandomForestRegressor(n_estimators=10000,
                            #    criterion='',
                               max_depth=None,
                               min_samples_split=2,
                               min_samples_leaf=1,
                               min_weight_fraction_leaf=0.0,
                               max_leaf_nodes=None,
                               min_impurity_decrease=0.0,
                               max_features="sqrt",
                               bootstrap=True,
                               oob_score=False,
                               n_jobs=-1,
                               random_state=42,
                               verbose=0,
                               warm_start=False,
                               ccp_alpha=0.0,
                               max_samples=None
                              )
forest.fit(X, y)

y_test_pred = forest.predict(test)
sample_submit = pd.read_csv("/content/content/MyDrive/Signate_data/2308自動車の走行距離予測/sample_submit.csv",header=None)
sub = pd.Series(y_test_pred)
sample_submit[1] = sub
sample_submit.to_csv("submit2.csv",header=None,index=False)

In [74]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print("Best Hyperparameters:", best_params)
print("Best RMSE:", np.sqrt(-grid_search.best_score_))


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors

Best Hyperparameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best RMSE: 3.0244370488325196
